#### This script uses the company name obtained earlier and tries to get the link of this company searching on google

In [ ]:
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import csv
import pandas as pd
import time
import datetime

In [ ]:
#import this to access shared functions allowing us to share code across notebooks
import sharedFunctions

In [ ]:
def findSearchResults(parsed_html):
    return parsed_html.body.find_all('div', attrs={'class':'r'})

In [ ]:
def getUrl(companyName):
    name = sharedFunctions.encodeString(companyName)
    url = 'https://www.google.com/search?q=' + name
    try:
        soup = sharedFunctions.downloadAndParseLink(url)
        items = findSearchResults(soup)
        link = sharedFunctions.getAttrFromTag(items[0].a, 'href')
        parsedUrl = urlparse(link)
        return '{uri.scheme}://{uri.netloc}/'.format(uri=parsedUrl)
    except Exception as ex:
        print("Some exception happened. ", ex)
        pass
    return ''    

### We will use the .csv created in the other notebook here to get the urls.

In [ ]:
csvData = pd.read_csv("JobInfoWithPostedDate.csv")

In [ ]:
companyLinks = []

In [ ]:
for index, row in csvData.iterrows():    
    postedOn = datetime.datetime.strptime(row["PostedOn"], "%Y-%m-%d")
    print("Getting for: {0} posted on {1}".format(row["CompanyName"], row["PostedOn"]))
    #We will only scrape jobs posted up to 15 days ago
    if((datetime.datetime.now() - datetime.timedelta(days=int(15))) < postedOn):            
        url = getUrl(row["CompanyName"] + ' canada')
        companyLinks.append(url)
        print("--:> " + url)
        time.sleep(2)
    else:
        print("Ignoring. Older than 15 days.")
        companyLinks.append('')

In [ ]:
csvData['CompanyLinks'] = companyLinks

In [ ]:
csvData.to_csv ('JobInfoWithPostedDate_WithLink.csv', index = False, header=True)